# config

In [ ]:
#|default_exp config

In [ ]:
#|hide
import nblite; from nblite import show_doc; nblite.nbl_export()

In [ ]:
#|export
import toml
from pydantic import BaseModel, field_validator
from typing import List, Dict, Union, Optional
from pathlib import Path

from nblite.const import code_loc_key_to_default_formats, nb_formats, nblite_config_file_name, format_to_file_exts, format_to_jupytext_format

In [ ]:
import nblite.config as this_module

In [ ]:
#|hide
show_doc(this_module.NBLiteConfig)

## NBLiteConfig

*Inherits from*: `BaseModel`

---

<h3>Methods</h3>

#### __post_process

```python
__post_process(self)
```

---

#### __init__

```python
__init__(self, **kwargs)
```

Create a new model by parsing and validating input data from keyword arguments.

Raises [`ValidationError`][pydantic_core.ValidationError] if the input data cannot be
validated to form a valid model.

`self` is explicitly positional-only to allow `self` as a field name.

---


In [ ]:
#|export
class ExportRule(BaseModel):
    from_key: str
    to_key: str

class CodeLocation(BaseModel):
    path: str
    format: str
    
    @property
    def file_ext(self) -> str:
        return format_to_file_exts[self.format]
    
    @property
    def jupytext_format(self) -> str:
        return format_to_jupytext_format[self.format]

class NBLiteConfig(BaseModel):
    """
    Configuration for the NBLite export pipeline.
    """
    export_pipeline: List[ExportRule]
    code_locations: Dict[str, CodeLocation]
    
    docs_cl: Optional[str] = None
    docs_title: Optional[str] = None
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.__post_process()
           
    @classmethod     
    def _has_cycle(cls, rules: List[ExportRule]) -> bool:
        # Create a graph from the export rules
        graph = {}
        for rule in rules:
            if rule.from_key not in graph:
                graph[rule.from_key] = []
            graph[rule.from_key].append(rule.to_key)

        visited = set()
        rec_stack = set()

        def dfs(node: str) -> bool:
            if node not in visited:
                visited.add(node)
                rec_stack.add(node)

                for neighbor in graph.get(node, []):
                    if neighbor not in visited and dfs(neighbor):
                        return True
                    elif neighbor in rec_stack:
                        return True

                rec_stack.remove(node)
            return False

        for node in graph:
            if dfs(node):
                return True

        return False
    
    @classmethod
    def _order_pipeline(cls, pipeline: List[ExportRule]) -> List[ExportRule]:
        ordered_pipeline = []
        _pipeline = list(pipeline)
        while _pipeline:
            next_found = False
            for rule in _pipeline:
                if all([r.to_key != rule.from_key for r in _pipeline]):
                    ordered_pipeline.append(rule)
                    _pipeline.remove(rule)
                    next_found = True
                    break
            if not next_found:
                raise RuntimeError("Cycle in pipeline")
        return ordered_pipeline
    
    def __post_process(self):
        for rule in self.export_pipeline:
            # Verify that all export rules from_keys and to_keys are in code_locations
            if rule.from_key not in self.code_locations:
                raise ValueError(f'"{rule.from_key}" not found in code_locations')
            if rule.to_key not in self.code_locations:
                raise ValueError(f'"{rule.to_key}" not found in code_locations')
            
            # Verify that no export rule is from a module to a notebook
            if self.code_locations[rule.from_key].format == "module":
                raise ValueError(f'Modules can only be exported to, not from.')
            
        for cl in self.code_locations.values():
            if cl.format not in nb_formats:
                raise ValueError(f'"{cl.format}" is not a valid format')
            
        if self._has_cycle(self.export_pipeline):
            raise ValueError("Export pipeline contains a cycle.")
        
        self.export_pipeline = self._order_pipeline(self.export_pipeline)

In [ ]:
conf = NBLiteConfig(
    docs_cl="nbs",
    export_pipeline=[
        ExportRule(from_key="nbs", to_key="pts"),
        ExportRule(from_key="pts", to_key="lib"),
        ExportRule(from_key="test_nbs", to_key="test_pts"),
        ExportRule(from_key="test_pts", to_key="test_lib"),
    ],
    code_locations={
        "nbs": CodeLocation(path="nbs", format="ipynb"),
        "pts": CodeLocation(path="pts", format="percent"),
        "test_nbs": CodeLocation(path="test_nbs", format="ipynb"),
        "test_pts": CodeLocation(path="test_pts", format="percent"),
        "lib": CodeLocation(path="my_module", format="module"),
        "test_lib": CodeLocation(path="test_module", format="module"),
    }
)

In [ ]:
#|hide
show_doc(this_module.parse_config_dict)

## parse_config_dict

```python
parse_config_dict(config_dict) -> NBLiteConfig
```

---


In [ ]:
#|export
def parse_config_dict(config_dict) -> NBLiteConfig:
    if 'export_pipeline' not in config_dict:
        raise ValueError("'export_pipeline' not found in config")

    # Change key name to match the Pydantic model
    if 'cl' in config_dict:
        config_dict['code_locations'] = config_dict.pop('cl')
    elif 'code_locations' not in config_dict:
        config_dict['code_locations'] = {}
        
    # Process the export pipeline
    if isinstance(config_dict['export_pipeline'], str):
        _export_pipeline = [rule.strip() for rule in config_dict['export_pipeline'].replace('\n', ',').split(',') if rule.strip()]
        config_dict['export_pipeline'] = [
            ExportRule(from_key=rule_str.strip().split('->')[0].strip(), to_key=rule_str.strip().split('->')[1].strip())
            for rule_str in _export_pipeline
            if rule_str.strip() and not rule_str.strip().startswith('#')
        ]
    else:
        config_dict['export_pipeline'] = [ExportRule(**rule) for rule in config_dict['export_pipeline']]
    
    # Add default code locations inferred from the export pipeline if they don't exist
    for rule in config_dict['export_pipeline']:
        if rule.from_key not in config_dict['code_locations'] and rule.from_key in code_loc_key_to_default_formats:
            config_dict['code_locations'][rule.from_key] = {}
        if rule.to_key not in config_dict['code_locations'] and rule.to_key in code_loc_key_to_default_formats:
            config_dict['code_locations'][rule.to_key] = {}

    # Fill in path and format of code locations, if possible to infer
    for code_loc_key, code_loc_config in config_dict['code_locations'].items():
        if 'path' not in code_loc_config:
            code_loc_config['path'] = code_loc_key
        if 'format' not in code_loc_config:
            if code_loc_key in code_loc_key_to_default_formats:
                code_loc_config['format'] = code_loc_key_to_default_formats[code_loc_key]
            else:
                raise ValueError(f"No default format for code location '{code_loc_key}'")
    
    config = NBLiteConfig(**config_dict)
    return config

In [ ]:
toml_string = '''
export_pipeline = """
    #nbs->pts
    pts ->lib
    test_nbs-> test_pts, test_pts->test_lib
"""
docs_cl = "nbs"

[cl.lib]
path = "my_module"
format = "module"

[cl.test_lib]
path = "test"

[cl.nbs]
format = "ipynb"

[cl.pts]
format = "percent"

[cl.test]
format = "module"

[cl.test_nbs]
format = "ipynb"

[cl.test_pts]
'''

parse_config_dict(toml.loads(toml_string)).model_dump()

{'export_pipeline': [{'from_key': 'pts', 'to_key': 'lib'},
  {'from_key': 'test_nbs', 'to_key': 'test_pts'},
  {'from_key': 'test_pts', 'to_key': 'test_lib'}],
 'code_locations': {'lib': {'path': 'my_module', 'format': 'module'},
  'test_lib': {'path': 'test', 'format': 'module'},
  'nbs': {'path': 'nbs', 'format': 'ipynb'},
  'pts': {'path': 'pts', 'format': 'percent'},
  'test': {'path': 'test', 'format': 'module'},
  'test_nbs': {'path': 'test_nbs', 'format': 'ipynb'},
  'test_pts': {'path': 'test_pts', 'format': 'percent'}},
 'docs_cl': 'nbs',
 'docs_title': None}

In [ ]:
toml_string = '''
export_pipeline = """
    pts ->lib
"""
docs_cl = "nbs"
'''

parse_config_dict(toml.loads(toml_string)).model_dump()

{'export_pipeline': [{'from_key': 'pts', 'to_key': 'lib'}],
 'code_locations': {'pts': {'path': 'pts', 'format': 'percent'},
  'lib': {'path': 'lib', 'format': 'module'}},
 'docs_cl': 'nbs',
 'docs_title': None}

In [ ]:
#|hide
show_doc(this_module._find_config_file)

## _find_config_file

```python
_find_config_file(curr_folder: Path) -> str
```

---


In [ ]:
#|exporti
def _find_config_file(curr_folder: Path) -> str:
    curr_folder = curr_folder.resolve()
    if (curr_folder / nblite_config_file_name).exists():
        return curr_folder / nblite_config_file_name
    else:
        if curr_folder.parent == curr_folder or curr_folder == Path('~').expanduser():
            return None
        else:
            return _find_config_file(curr_folder.parent)


In [ ]:
_find_config_file(Path('../../test_proj'))

Path('/Users/lukastk/dev-proj/nblite/test_proj/nblite.toml')

In [ ]:
#|hide
show_doc(this_module.read_config)

## read_config

```python
read_config(path) -> NBLiteConfig
```

---


In [ ]:
#|export
def read_config(path) -> NBLiteConfig:
    with open(path, 'r') as f:
        return parse_config_dict(toml.loads(f.read()))

In [ ]:
read_config('../../test_proj/nblite.toml')

NBLiteConfig(export_pipeline=[ExportRule(from_key='nbs', to_key='pcts'), ExportRule(from_key='pcts', to_key='lgts'), ExportRule(from_key='lgts', to_key='lib')], code_locations={'lib': CodeLocation(path='my_module', format='module'), 'pcts': CodeLocation(path='pcts', format='percent'), 'lgts': CodeLocation(path='lgts', format='light'), 'nbs': CodeLocation(path='nbs', format='ipynb')}, docs_cl='pcts', docs_title='nblite')

In [ ]:
#|hide
show_doc(this_module.get_project_root_and_config)

## get_project_root_and_config

```python
get_project_root_and_config(curr_folder: typing.Optional[pathlib.Path]) -> Path
```

---


In [ ]:
#|export
def get_project_root_and_config(curr_folder:Union[Path, None] = None) -> Path:
    curr_folder = Path(curr_folder) if curr_folder is not None else Path('.')
    if not curr_folder.exists(): raise ValueError(f"Directory '{curr_folder}' does not exist")
    config_path = _find_config_file(curr_folder)
    if config_path is None:
        raise ValueError("No nblite.toml found in the current or any parent directory")
    else:
        return config_path.parent, read_config(config_path)

In [ ]:
root_path, config = get_project_root_and_config('../../test_proj')
print(root_path)

/Users/lukastk/dev-proj/nblite/test_proj


In [ ]:
#|hide
#|hide
show_doc(this_module.get_top_level_code_locations)

## get_top_level_code_locations

```python
get_top_level_code_locations(config: NBLiteConfig) -> typing.List[str]
```

Returns the top level code locations in the export pipeline.

---


In [ ]:
#|export
def get_top_level_code_locations(config: NBLiteConfig) -> List[str]:
    """
    Returns the top level code locations in the export pipeline.
    """
    all_to_keys = set([rule.to_key for rule in config.export_pipeline])
    return [rule.from_key for rule in config.export_pipeline if rule.from_key not in all_to_keys]

In [ ]:
root_path, config = get_project_root_and_config('../../test_proj')
get_top_level_code_locations(config)

['nbs']

In [ ]:
#|hide
show_doc(this_module.get_downstream_module)

## get_downstream_module

```python
get_downstream_module(config: NBLiteConfig, starting_code_loc_key: str) -> str
```

Finds the first downstream code location in the export pipeline that is of format 'module'.

**Arguments:**
- `config` (*NBLiteConfig*): The configuration object containing the export pipeline and code locations.
- `starting_code_loc_key` (*str*): The key of the starting code location.

**Returns:** *str*: The key of the first downstream code location with format 'module', or None if not found.

---


In [ ]:
#|exporti
def get_downstream_module(config: NBLiteConfig, starting_code_loc_key: str) -> str:
    """
    Finds the first downstream code location in the export pipeline that is of format 'module'.

    Args:
        config (NBLiteConfig): The configuration object containing the export pipeline and code locations.
        starting_code_loc_key (str): The key of the starting code location.

    Returns:
        str: The key of the first downstream code location with format 'module', or None if not found.

    Raises:
        ValueError: If the starting code location key is not found in the configuration.
    """
    def get_next_cl(curr_cl: str) -> ExportRule:
        for rule in config.export_pipeline:
            if rule.from_key == curr_cl:
                return rule.to_key
        return None
    
    if not starting_code_loc_key in config.code_locations:
        raise ValueError(f"Starting code location '{starting_code_loc_key}' not found in config.")
    
    curr_cl = starting_code_loc_key
    while curr_cl is not None:
        if config.code_locations[curr_cl].format == "module":
            return curr_cl
        curr_cl = get_next_cl(curr_cl)
    
    return None


In [ ]:
config = read_config('../../test_proj/nblite.toml')
get_downstream_module(config, 'nbs')

'lib'